In [1]:
from transformers import pipeline
    
classifier = pipeline("text-classification", model="./model_local", truncation=True)


2024-04-22 00:36:11.214142: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-22 00:36:11.269324: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-22 00:36:13.024672: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
classifier.model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [3]:
import pandas as pd
df = pd.read_csv('clean_test_or_not_test_debt.csv',index_col = 0)
df.head()

,text_clean,label
0,make sure integration tests are running with i...,0
1,cover anvil service response in unit testshttp...,0
2,service apprepositorycontroller should be cach...,0
3,many caches persist between test casesan examp...,0
4,improve friendliness of behat text pattern mat...,0


In [4]:
def classify_text(text):
    # Get prediction
    result = classifier(text)
    # Return the label of the highest scoring classification
    return result[0]['label']

# Apply the classification function to your text column (assuming it's named 'text_clean')
df['predicted_label'] = df['text_clean'].apply(classify_text)


In [5]:
df.head(100)

,text_clean,label,predicted_label
0,make sure integration tests are running with i...,0,LABEL_0
1,cover anvil service response in unit testshttp...,0,LABEL_0
2,service apprepositorycontroller should be cach...,0,LABEL_1
3,many caches persist between test casesan examp...,0,LABEL_0
4,improve friendliness of behat text pattern mat...,0,LABEL_0
...,...,...,...
95,ci test flake can modify upstreams repeatedly ...,0,LABEL_0
96,fix broken shebang on smoke test script contin...,0,LABEL_0
97,add webworkers tests description add unit test...,0,LABEL_0
98,servlet 60 correct the list of valid orderings...,0,LABEL_0


In [6]:
from sklearn.metrics import accuracy_score, f1_score

# Assuming your predicted labels are in a format like 'LABEL_0', 'LABEL_1', etc.
# Convert these to numeric by extracting the number part and converting it to an integer
df['predicted_label_numeric'] = df['predicted_label'].apply(lambda x: int(x.split('_')[1]))

# Now, your actual labels are assumed to be in the 'label' column and already numeric
actual_labels = df['label']

# Predicted labels are now in 'predicted_label_numeric'
predicted_labels = df['predicted_label_numeric']


df.head()

,text_clean,label,predicted_label,predicted_label_numeric
0,make sure integration tests are running with i...,0,LABEL_0,0
1,cover anvil service response in unit testshttp...,0,LABEL_0,0
2,service apprepositorycontroller should be cach...,0,LABEL_1,1
3,many caches persist between test casesan examp...,0,LABEL_0,0
4,improve friendliness of behat text pattern mat...,0,LABEL_0,0


In [7]:
df.to_csv('TestNotTestRes_bert.csv', index=False)

In [8]:
# Calculate accuracy
accuracy = accuracy_score(actual_labels, predicted_labels)

# Calculate F1 score
# Assuming it's a binary classification, you could specify the average method if it's multi-class
f1 = f1_score(actual_labels, predicted_labels, average='binary' if df['label'].nunique() == 2 else 'weighted')

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Accuracy: 0.7285714285714285
F1 Score: 0.6563307493540051
